In [1]:
!pip install geopy


In [2]:
import pandas as pd
import numpy as np

# 1) CSV dosyasını oku
df_new = pd.read_csv("df_new.csv")

print("df_new CSV'den başarıyla okundu.")
print(df_new.head())

# 2) Tarih sütununu datetime tipine çevir
df_new['FlightDate'] = pd.to_datetime(df_new['FlightDate'], errors='coerce')

# 3) CRSDepTime_time veya CRSArrTime_time sütunlarındaki veriyi
#    '1987-10-01' + '14:30:00' şeklinde birleştirecek fonksiyon
def combine_datetime(date_val, time_str):
    if pd.isna(time_str) or str(time_str).lower() in ["nan", ""]:
        return pd.NaT  # Eksik/NaN zaman
    # Aksi halde birleştir
    return pd.to_datetime(f"{date_val.date()} {time_str}", errors='coerce')

# 4) Kalkış ve varış için tam datetime sütunları oluştur
df_new['CRSDepDateTime'] = df_new.apply(
    lambda row: combine_datetime(row['FlightDate'], row['CRSDepTime_time']),
    axis=1
)
df_new['CRSArrDateTime'] = df_new.apply(
    lambda row: combine_datetime(row['FlightDate'], row['CRSArrTime_time']),
    axis=1
)

# 5) Saatlik yuvarlama
df_new['DepHour'] = df_new['CRSDepDateTime'].dt.floor('h')
df_new['ArrHour'] = df_new['CRSArrDateTime'].dt.floor('h')

print("Uçuş verisi son durumu:")
print(df_new[['OriginCityName','CRSDepDateTime','DepHour','CRSArrDateTime','ArrHour']].head())


/tmp/ipykernel_1470/960910292.py:5: DtypeWarning: Columns (20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df_new = pd.read_csv("df_new.csv")


df_new CSV'den başarıyla okundu.
   Year  Month  DayofMonth  DayOfWeek  FlightDate   OriginCityName  \
0  1987     10           1          4  1987-10-01      Houston, TX   
1  1987     10           1          4  1987-10-01       Dallas, TX   
2  1987     10           1          4  1987-10-01      Kinston, NC   
3  1987     10           1          4  1987-10-01  Los Angeles, CA   
4  1987     10           1          4  1987-10-01    Charlotte, NC   

      DestCityName  DepDel15  DepartureDelayGroups DepTimeBlk  ...  \
0       Newark, NJ       0.0                   0.0  1300-1359  ...   
1      Houston, TX       0.0                   0.0  0700-0759  ...   
2    Baltimore, MD       0.0                   0.0  1000-1059  ...   
3  Minneapolis, MN       0.0                   0.0  0001-0559  ...   
4      Atlanta, GA       0.0                   0.0  1600-1659  ...   

   ElapsedTimeDiff     Speed TaxiIn_log  DepDelayMinutes_log  TaxiOut_log  \
0             13.0  6.829268        NaN         

In [3]:
# "Year", "Month", vb. sütunları değil, "FlightDate" gibi gerçek datetime sütununu kullanıyoruz
df_new['FlightDate'] = pd.to_datetime(df_new['FlightDate'], errors='coerce')

# Gruplayarak min ve max tarihleri bulalım
min_max_dates = (
    df_new
    .groupby('OriginCityName')['FlightDate']
    .agg(['min', 'max'])  # min, max
    .reset_index()
    .rename(columns={'min': 'StartDate', 'max': 'EndDate'})
)

print(min_max_dates.head(20))


                    OriginCityName  StartDate    EndDate
0                     Aberdeen, SD 2012-05-06 2019-12-12
1                      Abilene, TX 2001-01-08 2019-12-01
2                  Adak Island, AK 2003-08-03 2016-09-22
3                    Aguadilla, PR 1993-11-21 2019-07-06
4                        Akron, OH 1987-10-15 2020-03-25
5                       Albany, GA 2003-09-19 2020-01-10
6                       Albany, NY 1987-10-01 2020-01-20
7                  Albuquerque, NM 1987-10-01 2020-03-20
8                   Alexandria, LA 2003-02-10 2019-09-13
9   Allentown/Bethlehem/Easton, PA 1987-10-19 2019-12-05
10                      Alpena, MI 2016-02-02 2018-05-02
11                    Amarillo, TX 1987-12-17 2020-02-28
12                   Anchorage, AK 1987-10-23 2020-03-23
13                    Appleton, WI 1988-04-23 2020-03-12
14               Arcata/Eureka, CA 2003-03-10 2019-10-05
15                   Asheville, NC 1987-10-07 2020-03-17
16                     Ashland,

In [4]:
import time
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="flight_weather_app")

def geocode_city(city_name):
    """
    city_name -> (latitude, longitude)
    """
    try:
        loc = geolocator.geocode(city_name)
        time.sleep(1)  # rate limit
        if loc:
            return (loc.latitude, loc.longitude)
        else:
            return (None, None)
    except:
        return (None, None)

# min_max_dates tablosundan her şehrin adını alarak geocode et
coords_by_city = {}

for idx, row in min_max_dates.iterrows():
    city = row['OriginCityName']
    lat, lon = geocode_city(city)
    coords_by_city[city] = (lat, lon)

print("Koordinatlar:")
for c, (lat, lon) in coords_by_city.items():
    print(c, lat, lon)


Koordinatlar:
Aberdeen, SD 45.4649805 -98.487813
Abilene, TX 32.44645 -99.7475905
Adak Island, AK 51.79622495 -176.57442259513414
Aguadilla, PR 18.449655149999998 -67.11847643508234
Akron, OH 41.083064 -81.518485
Albany, GA 31.5782062 -84.1556809
Albany, NY 42.6511674 -73.754968
Albuquerque, NM 35.0841034 -106.650985
Alexandria, LA 31.3119463 -92.4453558
Allentown/Bethlehem/Easton, PA 40.651113699999996 -75.44223357287248
Alpena, MI 45.0176181 -83.6670019
Amarillo, TX 35.20729 -101.8371192
Anchorage, AK 61.2163129 -149.894852
Appleton, WI 44.2613967 -88.4069744
Arcata/Eureka, CA 40.9763645 -124.10842589937442
Asheville, NC 35.595363 -82.5508407
Ashland, WV 37.4084488 -81.3526017
Aspen, CO 39.1911128 -106.82356
Atlanta, GA 33.7489924 -84.3902644
Atlantic City, NJ 39.3642852 -74.4229351
Augusta, GA 33.4709714 -81.9748429
Austin, TX 30.2711286 -97.7436995
Bakersfield, CA 35.3738712 -119.019463
Baltimore, MD 39.2908816 -76.610759
Bangor, ME 44.8016255 -68.7713289
Barrow, AK 71.3865172 -156

In [ ]:
import requests
import time
import pandas as pd

def get_open_meteo_archive_data(lat, lon, start_date, end_date):
    """
    start_date, end_date: 'YYYY-MM-DD' formatında string
    Open-Meteo Archive ERA5: 1979 – günümüz
    """
    url = (
        f"https://archive-api.open-meteo.com/v1/era5?"
        f"latitude={lat}&longitude={lon}"
        f"&start_date={start_date}&end_date={end_date}"
        f"&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m,precipitation,weathercode"
        f"&timezone=auto"
    )
    try:
        response = requests.get(url, timeout=15)
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Arşiv API isteği başarısız (HTTP {response.status_code}): {url}")
            return None
    except Exception as e:
        print(f"İstek hatası: {e}")
        return None

# Burada, min_max_dates veri çerçevenizde
# her satır: [ OriginCityName, StartDate, EndDate ]
# Koordinatlar coords_by_city[city] = (lat, lon)
# Rate limit'i aşmamak için her istekten sonra 5 sn bekleyeceğiz.
# 10–20 şehir sonrası 60 sn gibi daha uzun bekleme eklemeyi de düşünebilirsiniz.

weather_rows = []
city_counter = 0  # kaç şehir işlediğimizi izlemek için

for idx, row in min_max_dates.iterrows():
    city = row['OriginCityName']
    start_dt = row['StartDate']
    end_dt = row['EndDate']

    # coords_by_city sözlüğünden lat/lon al
    lat, lon = coords_by_city[city]  
    if (not lat) or (not lon):
        print(f"Koordinat bulunamadı: {city}, atlanıyor.")
        continue

    print(f"[Şehir] {city} için veri çekiyoruz:")
    print(f"  - Tarih aralığı: {start_dt} – {end_dt}\n")

    # 1) Tarih aralığı büyükse 1 yıllık bloklar
    current_start = start_dt
    while current_start <= end_dt:
        next_year = current_start.year + 1
        # örn. 1987-12-31
        block_end = pd.to_datetime(f"{next_year}-01-01") - pd.Timedelta(days=1)
        if block_end > end_dt:
            block_end = end_dt

        # API'ye göndereceğimiz string formatı
        s_str = current_start.strftime('%Y-%m-%d')
        e_str = block_end.strftime('%Y-%m-%d')

        print(f"  => Tarih bloğu: {s_str} – {e_str}")

        # 2) API çağrısı yap
        data = get_open_meteo_archive_data(lat, lon, s_str, e_str)
        if data and 'hourly' in data:
            hour_data = data['hourly']
            for i, t_str in enumerate(hour_data['time']):
                weather_rows.append({
                    'City': city,
                    'DateTimeHour': pd.to_datetime(t_str),
                    'Temperature': hour_data['temperature_2m'][i],
                    'Humidity': hour_data['relative_humidity_2m'][i],
                    'WindSpeed': hour_data['wind_speed_10m'][i],
                    'Precipitation': hour_data['precipitation'][i],
                    'WeatherCode': hour_data['weathercode'][i]
                })

        # 3) Bir sonraki bloğa geçmeden önce 5 saniye bekle (rate limit!)
        time.sleep(5)

        # 4) Bloğu tamamladıktan sonra start'ı kaydır
        current_start = block_end + pd.Timedelta(days=1)
        if current_start > end_dt:
            break

    city_counter += 1

    # Örnek: Her 5 şehirde bir, 60 sn bekleme (büyük veri çekerken 429'u azaltmak için)
    if city_counter % 5 == 0:
        print("5 şehir işlendi, 1 dakika bekliyoruz...\n")
        time.sleep(60)

# 5) Tüm şehirlerin verilerini tek DataFrame'e at
weather_df = pd.DataFrame(weather_rows)
print("\n--- Archive Hava Durumu boyutu:", weather_df.shape)
print(weather_df.head(10))


[Şehir] Aberdeen, SD için veri çekiyoruz:
  - Tarih aralığı: 2012-05-06 00:00:00 – 2019-12-12 00:00:00

  => Tarih bloğu: 2012-05-06 – 2012-12-31
  => Tarih bloğu: 2013-01-01 – 2013-12-31
  => Tarih bloğu: 2014-01-01 – 2014-12-31
  => Tarih bloğu: 2015-01-01 – 2015-12-31
  => Tarih bloğu: 2016-01-01 – 2016-12-31
  => Tarih bloğu: 2017-01-01 – 2017-12-31
  => Tarih bloğu: 2018-01-01 – 2018-12-31
  => Tarih bloğu: 2019-01-01 – 2019-12-12
[Şehir] Abilene, TX için veri çekiyoruz:
  - Tarih aralığı: 2001-01-08 00:00:00 – 2019-12-01 00:00:00

  => Tarih bloğu: 2001-01-08 – 2001-12-31
  => Tarih bloğu: 2002-01-01 – 2002-12-31
  => Tarih bloğu: 2003-01-01 – 2003-12-31
  => Tarih bloğu: 2004-01-01 – 2004-12-31
  => Tarih bloğu: 2005-01-01 – 2005-12-31
  => Tarih bloğu: 2006-01-01 – 2006-12-31
  => Tarih bloğu: 2007-01-01 – 2007-12-31
  => Tarih bloğu: 2008-01-01 – 2008-12-31
  => Tarih bloğu: 2009-01-01 – 2009-12-31
  => Tarih bloğu: 2010-01-01 – 2010-12-31
  => Tarih bloğu: 2011-01-01 – 2011-1

In [ ]:
df_merged_origin = pd.merge(
    df_new,     # Uçuş verisi
    weather_df, # Hava durumu
    how='left',
    left_on=['OriginCityName', 'DepHour'],
    right_on=['City', 'DateTimeHour']
)

df_merged_origin.rename(columns={
    'Temperature': 'Origin_Temp',
    'Humidity': 'Origin_Humidity',
    'WindSpeed': 'Origin_WindSpeed',
    'Precipitation': 'Origin_Precip',
    'WeatherCode': 'Origin_WeatherCode'
}, inplace=True)

# Artık City, DateTimeHour sütunları gereksiz, drop edebiliriz
df_merged_origin.drop(['City', 'DateTimeHour'], axis=1, inplace=True)


In [ ]:
df_final = pd.merge(
    df_merged_origin,
    weather_df,
    how='left',
    left_on=['DestCityName', 'ArrHour'],
    right_on=['City', 'DateTimeHour']
)

df_final.rename(columns={
    'Temperature': 'Dest_Temp',
    'Humidity': 'Dest_Humidity',
    'WindSpeed': 'Dest_WindSpeed',
    'Precipitation': 'Dest_Precip',
    'WeatherCode': 'Dest_WeatherCode'
}, inplace=True)

df_final.drop(['City', 'DateTimeHour'], axis=1, inplace=True)

print("Son merge bitti, df_final boyutu:", df_final.shape)
print(df_final.head(10))


In [1]:
# df_new DataFrame'inizi CSV formatında kaydedin
weather_df.to_csv("weather_df.csv", index=False)

print("weather_df CSV dosyası olarak kaydedildi.")


NameError: name 'weather_df' is not defined